# Libraries

In [ ]:
import pandas as pd
import pickle
from scipy.stats import poisson
from bs4 import BeautifulSoup
import requests
from datetime import datetime

# Get data from Wikipedia

In [ ]:
def get_matches(year, web):
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football

## FIFA Women's World Cup

In [ ]:
years_wk = [1991,1995,1999,2003,2007,2011,2015,2019]

fifa = [get_matches(year, f'https://en.wikipedia.org/wiki/{year}_FIFA_Women%27s_World_Cup') for year in years_wk]
df_fifa = pd.concat(fifa, ignore_index=True)
df_historical_data = df_fifa
df_historical_data

## UEFA Women's Euro


In [ ]:
years_euro = [1984,1987,1989,1991,1993,1995,1997,2001,2005,2009,2013,2017,2022]

euro = [get_matches(year, f'https://en.wikipedia.org/wiki/UEFA_Women%27s_Euro_{year}') for year in years_euro]
df_euro = pd.concat(euro, ignore_index=True)
df_historical_data_euro = df_euro

## Copa America

In [ ]:
years_cp = [1991,1995,1998,2003,2006,2010,2014,2018,2022]

cp_america = [get_matches(year, f'https://en.wikipedia.org/wiki/{year}_South_American_Women%27s_Football_Championship') for year in years_cp]
df_cp_america = pd.concat(cp_america, ignore_index=True)
df_historical_data_cp_america = df_cp_america

## CONCACAF

In [ ]:
years_conc = [1991,1993,1994,1998,2000,2002,2006,2010,2014,2018,2022]

concacaf = [get_matches(year, f'https://en.wikipedia.org/wiki/{year}_CONCACAF_Women%27s_Championship') for year in years_conc]
df_concacaf = pd.concat(concacaf, ignore_index=True)
df_historical_data_concacaf = df_concacaf

## She Belives Cup

In [ ]:
years_sb = [2016,2017,2018,2019,2020,2021,2022,2023]

sheBelives_cup = [get_matches(year, f'https://en.wikipedia.org/wiki/{year}_SheBelieves_Cup') for year in years_sb]
df_sheBelives_cup = pd.concat(sheBelives_cup, ignore_index=True)
df_historical_data_sheBelives_cup = df_sheBelives_cup

## Finalissima

In [ ]:
years_fina = [2023]

finalissima = [get_matches(year, f'https://en.wikipedia.org/wiki/{year}_Women%27s_Finalissima') for year in years_fina]
df_finalissima = pd.concat(finalissima, ignore_index=True)
df_historical_data_finalissima = df_finalissima

## Women's Olympic

In [ ]:
years_olympic = [1996,2000,2004,2008,2012,2016,2020]

olympic = [get_matches(year, f'https://en.wikipedia.org/wiki/Football_at_the_{year}_Summer_Olympics_%E2%80%93_Women%27s_tournament') for year in years_olympic]
df_olympic = pd.concat(olympic, ignore_index=True)
df_historical_data_olympic = df_olympic

# Kaggle Data

In [ ]:
df_historical_data = pd.read_csv('/content/results.csv')

In [ ]:
df_historical_data['date'] = df_historical_data.date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())
df_historical_data['Year'] = df_historical_data.date.apply(lambda x: x.strftime("%Y"))

In [ ]:
df_historical_data.rename(columns={'home_team': 'HomeTeam', 'away_team': 'AwayTeam',
                                   'home_score':'HomeGoals', 'away_score':'AwayGoals'}, inplace=True)

In [ ]:
df_historical_data

# Clean Data

In [ ]:
df_missing_data = pd.concat([df_historical_data_euro,
                             df_historical_data_cp_america,
                             df_historical_data_finalissima,
                             df_historical_data_sheBelives_cup,
                             df_historical_data_concacaf,
                             df_historical_data_olympic], ignore_index=True)

In [ ]:
df_missing_data.dropna(inplace=True)
df_historical_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_data.drop_duplicates(inplace=True)
df_historical_data.sort_values('year', inplace=True)
df_historical_data

,home,score,away,year
284,England,2–1,Denmark,1984
285,Italy,2–3,Sweden,1984
286,Denmark,0–1,England,1984
287,Sweden,2–1,Italy,1984
288,Sweden,1–0,England,1984
...,...,...,...,...
901,Australia,3–0,Jamaica,2023
599,England,1–1,Brazil,2023
642,Japan,0–1,Brazil,2023
885,Wales,1–0 (a.e.t.),Bosnia and Herzegovina,2023


In [ ]:
# deleting match with problem
delete_index = df_historical_data[df_historical_data['home'].str.contains('Canada') &
                                  df_historical_data['away'].str.contains('Trinidad and Tobago')].index

df_historical_data.drop(index=delete_index, inplace=True)

df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d–]', '', regex=True)
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()

df_historical_data

,home,score,away,year
284,England,2–1,Denmark,1984
285,Italy,2–3,Sweden,1984
286,Denmark,0–1,England,1984
287,Sweden,2–1,Italy,1984
288,Sweden,1–0,England,1984
...,...,...,...,...
901,Australia,3–0,Jamaica,2023
599,England,1–1,Brazil,2023
642,Japan,0–1,Brazil,2023
885,Wales,1–0,Bosnia and Herzegovina,2023


In [ ]:
# splitting score columns into home and away goals and dropping score column
df_historical_data[['HomeGoals', 'AwayGoals']] = df_historical_data['score'].str.split('–', expand=True)
df_historical_data.drop('score', axis=1, inplace=True)

# renaming columns and changing data types
df_historical_data.rename(columns={'home': 'HomeTeam', 'away': 'AwayTeam',
                                   'year':'Year'}, inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']
df_historical_data = df_historical_data.replace({"Australia": "Australia (H)"})
df_historical_data = df_historical_data.replace({"New Zealand": "New Zealand (H)"})

df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
284,England,Denmark,1984,2,1,3
285,Italy,Sweden,1984,2,3,5
286,Denmark,England,1984,0,1,1
287,Sweden,Italy,1984,2,1,3
288,Sweden,England,1984,1,0,1
...,...,...,...,...,...,...
901,Australia (H),Jamaica,2023,3,0,3
599,England,Brazil,2023,1,1,2
642,Japan,Brazil,2023,0,1,1
885,Wales,Bosnia and Herzegovina,2023,1,0,1


In [ ]:
df_historical_data.to_csv('df_historical_data.csv',index=False)

# Kaggle dataset Clean

In [ ]:
df_historical_data_kaggle = pd.DataFrame(df_historical_data, columns=['HomeTeam','AwayTeam', 'Year', 'HomeGoals', 'AwayGoals'])

In [ ]:
df_historical_data_kaggle

In [ ]:
df_historical_data_kaggle = df_historical_data_kaggle.astype({'HomeGoals': int, 'AwayGoals':int, 'Year': int})

# creating new column "totalgoals"
df_historical_data_kaggle['TotalGoals'] = df_historical_data_kaggle['HomeGoals'] + df_historical_data_kaggle['AwayGoals']

df_historical_data_kaggle

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,Italy,France,1969,1,0,1
1,Denmark,England,1969,4,3,7
2,England,France,1969,2,0,2
3,Italy,Denmark,1969,3,1,4
4,England,West Germany,1970,5,1,6
...,...,...,...,...,...,...
5590,United States,Brazil,2023,2,1,3
5591,Paraguay,Panama,2023,0,1,1
5592,Taiwan,Papua New Guinea,2023,5,0,5
5593,New Zealand (H),Argentina,2023,0,1,1


In [ ]:
df_historical_data_kaggle.to_csv('df_historical_data_kaggle.csv',index=False)

# Get Games from 2023 Women's Word Cup

In [ ]:
df_future = get_matches(2023, f'https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')

In [ ]:
df_future.to_csv('df_future.csv',index=False)